```bash
wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh

chmod +x Miniconda3-latest-Linux-x86_64.sh

./Miniconda3-latest-Linux-x86_64.sh

Please, press ENTER to continue
```

Einmal Leertaste drücken.

```bash
Please answer 'yes' or 'no':'
>>> yes

Press ENTER to confirm the location

Do you wish the installer to initialize Miniconda3
by running conda init? [yes|no]
[no] >>> yes

conda config --set auto_activate_base false

conda create -n python_workshop_env python=3

Proceed ([y]/n)? y

. ~/.bashrc

conda activate python_workshop_env

conda install s3fs bokeh dask ipykernel numpy pandas "dask[distributed]" mpi4py

Proceed ([y]/n)? y

conda install -c conda-forge dask-mpi
```


In [3]:
#import os

#f = open(os.path.expanduser("~/job_dask_mpi.sh"), "w")
#f.write("""#!/bin/bash -l
#conda activate python_workshop_env
#mpirun -np 20 dask-mpi --scheduler-file ~/dask-scheduler.json --interface='ib0' --local-directory='/tmp'""")
#f.close()

```bash
tar -zcvf ~/miniconda3/envs/python_workshop_env.gz -C ~/miniconda3/envs/ python_workshop_env/
```

In [15]:
import os

f = open(os.path.expanduser("~/job_dask_mpi.sh"), "w")
f.write("""#!/bin/bash -l
NODES=$(scontrol show hostname | cat)
for NODE in $NODES
do
    srun -N 1 -n 1 -w $NODE --pty /bin/bash -c "mkdir /tmp/envs \
    && cp ~/miniconda3/envs/python_workshop_env.gz /tmp/envs && \
    tar -zxvf /tmp/envs/python_workshop_env.gz --directory /tmp/envs" &
done

wait

conda activate /tmp/envs/python_workshop_env

mpirun -np 9 dask-mpi --scheduler-file ~/dask-scheduler.json --interface='ib0' --local-directory='/tmp' --no-nanny""")
f.close()

In [16]:
os.system("sinfo_t_idle")

Partition dev_single    :      5 nodes idle
Partition single        :     20 nodes idle
Partition dev_multiple  :      4 nodes idle
Partition multiple      :      1 nodes idle
Partition fat           :      2 nodes idle
Partition dev_multiple_e:      8 nodes idle
Partition multiple_e    :      0 nodes idle
Partition dev_special   :      2 nodes idle
Partition special       :      0 nodes idle
Partition gpu_4         :      0 nodes idle
Partition dev_gpu_4     :      1 nodes idle
Partition gpu_8         :      0 nodes idle


0


```bash
rm -f ~/dask-scheduler.json  && sbatch -p dev_multiple --nodes=2 --ntasks=9 --ntasks-per-node=5 --time=30:00 --mem=90000mb ./job_dask_mpi.sh
```

In [17]:
import os

os.system("rm -f ~/dask-scheduler.json  && sbatch -p dev_multiple --nodes=2 --ntasks=9 --ntasks-per-node=5 --time=30:00 --mem=90000mb ~/job_dask_mpi.sh")
#TODO: warum bricht der Job über os.system immer nach <1min. ab?

Submitted batch job 20146874


0

In [18]:
os.system("squeue --start")

             JOBID PARTITION     NAME     USER ST          START_TIME  NODES SCHEDNODES           NODELIST(REASON)
          20146874 dev_multi job_dask es_pkoes PD                 N/A      2 (null)               (None)


0

In [19]:
os.system("squeue")

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
          20146874 dev_multi job_dask es_pkoes PD       0:00      2 (None)
          20146778    single jupyterh es_pkoes  R      21:05      1 uc2n365


0

In [20]:
import os, time

while not os.path.isfile(os.path.expanduser("~/dask-scheduler.json")):
    time.sleep(1)

In [21]:
from dask.distributed import Client
client = Client(scheduler_file=os.path.expanduser('~/dask-scheduler.json'))

In [22]:
#from dask.distributed import Client
#client = Client('172.26.20.1:40687')

In [23]:
client

<Client: 'tcp://172.26.20.5:42251' processes=8 threads=640, memory=703.12 GiB>

In [6]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
import datetime
import s3fs

In [9]:
%%time

taxi = dd.read_csv(
    's3://nyc-tlc/trip data/yellow_tripdata_2019-*.csv',
    assume_missing=True, #Beim Einlesen werden alle Ints zu Floats. Dies erlaubt fehlende Werte
    parse_dates=['tpep_pickup_datetime', 'tpep_dropoff_datetime'], #Interpretiert diese Spalten als Datum
    storage_options={'anon': True}, #Für S3: KeineAuthentifizierung für diesen Bucket nötig
)

print(f"Row count: {len(taxi)}") #Zeilen aller Datensätze zusammen
print(f"Size in GB: {taxi.memory_usage(deep=True).sum().compute() / 1e9}")

Row count: 84399019
Size in GB: 16.367014316
CPU times: user 216 ms, sys: 23.6 ms, total: 240 ms
Wall time: 2min 6s


2min 21s
2min
2min 14s
2min 7s
2min 11s

2min 13s
2min 3s
2min 6s

In [24]:
import dask.array as da
x = da.random.random((100000,100000), chunks="16 MiB")
x

dask.array<random_sample, shape=(100000, 100000), dtype=float64, chunksize=(1250, 1250), chunktype=numpy.ndarray>

In [25]:
y = (x + x.T) - x.mean(axis=0)

In [29]:
%%time

y.sum().compute()

CPU times: user 2.85 s, sys: 99.6 ms, total: 2.95 s
Wall time: 19.9 s


4999992725.603169

19.9 s
18.4 s

29.7 s
22.8 s
20.6 s
19.9 s